In [1]:
import json
import requests
import pandas as pd
import time
import pymysql
from multiprocessing import Pool
from sqlalchemy import create_engine

In [2]:
#https://www.nasdaq.com/screening/companies-by-industry.aspx?industry=Technology&exchange=NASDAQ
#for stock tickers and companies informaiton

file_to_load = "Resources/companylist.csv"
stocks = pd.read_csv(file_to_load)

In [3]:
stocks = stocks[['Symbol','Name','MarketCap','IPOyear','Sector','Industry']]

In [4]:
stocks = stocks.rename(columns={'Name':'Company_Name'})

In [5]:
stocks

,Symbol,Company_Name,MarketCap,IPOyear,Sector,Industry
0,GOOG,Alphabet Inc.,8.049285e+11,2004.0,Technology,"Computer Software: Programming, Data Processing"
1,GOOGL,Alphabet Inc.,8.098504e+11,NaN,Technology,"Computer Software: Programming, Data Processing"
2,AAPL,Apple Inc.,8.229107e+11,1980.0,Technology,Computer Manufacturing
3,CSCO,"Cisco Systems, Inc.",2.274528e+11,1990.0,Technology,Computer Communications Equipment
4,FB,"Facebook, Inc.",4.923418e+11,2012.0,Technology,"Computer Software: Programming, Data Processing"
5,INTC,Intel Corporation,2.381162e+11,NaN,Technology,Semiconductors
6,MSFT,Microsoft Corporation,8.573699e+11,1986.0,Technology,Computer Software: Prepackaged Software


In [6]:
stock_tickers = stocks["Symbol"]

In [7]:
stock_tickers.count()

7

In [ ]:
#query_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=GOOG&outputsize=full&apikey=9U1TJY7GWT8RKKHT"
#response = requests.get(query_url).json()
#trading_data = response['Time Series (Daily)']
#df = pd.DataFrame.from_records(trading_data).T
#df.reset_index(inplace=True)
#df['ticker'] = 'GOOG'
#df.head()


In [8]:
# I found https://www.alphavantage.co/ provides free API for stock trading data.
# In addition, below codes are for initial extraction of trading data based on daily prices.
# For daily update purposes/change of trading data freqencies (from mins to monthly), user needs to update 'qurey_url'
# accordingly by going on to Alpha Avantage's website and review its documentation.

i = 0
trading_data = []
master_df = pd.DataFrame()

#create a for loop for getting trading data of each stock
for stock in stock_tickers:
    query_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock}&outputsize=full&apikey=9U1TJY7GWT8RKKHT"
    response = requests.get(query_url).json()
    try:
        print(f"{i}. Getting information for {stock}.")
        #append daily trading data into trading_data list
        trading_data.append(response['Time Series (Daily)'])
        #pause for 10 s to avoid being disconnected from alpha's server
        time.sleep(10.0)
        print(f'------------------------------------------')
        #create a dataframe and store trading data in trading_data list
        records = pd.DataFrame.from_records(trading_data[i]).T
        #rest the dataframe and assign index to date column
        records.reset_index(inplace=True)
        #add a new column with the stock ticker
        records['Ticker'] = stock
        # append each stock's dataframe to master_df
        master_df = master_df.append(records, ignore_index=True)
    except (KeyError):
        print("Missing field/result... skipping.")
    i += 1
    print(f'Moving on to the next stock in the list...')
    print(f'------------------------------------------')


0. Getting information for GOOG.
------------------------------------------
Moving on to the next stock in the list...
------------------------------------------
1. Getting information for GOOGL.
------------------------------------------
Moving on to the next stock in the list...
------------------------------------------
2. Getting information for AAPL.
------------------------------------------
Moving on to the next stock in the list...
------------------------------------------
3. Getting information for CSCO.
------------------------------------------
Moving on to the next stock in the list...
------------------------------------------
4. Getting information for FB.
------------------------------------------
Moving on to the next stock in the list...
------------------------------------------
5. Getting information for INTC.
------------------------------------------
Moving on to the next stock in the list...
------------------------------------------
6. Getting information for MS

In [9]:
#rename column
master_df = master_df.rename(columns={'index':'Trading_Date','1. open':'Open_Price',
                                     '2. high':'Higest_Price','3. low':'Lowest_Price',
                                     '4. close': 'Close_Price', '5. volume': 'Trading_Volume',
                                     })

In [10]:
master_df.head()

,Trading_Date,Open_Price,Higest_Price,Lowest_Price,Close_Price,Trading_Volume,Ticker
0,2014-03-27,568.0000,568.0000,552.9200,558.4600,13100,GOOG
1,2014-03-28,561.2000,566.4300,558.6700,559.9900,41100,GOOG
2,2014-03-31,566.8900,567.0000,556.9300,556.9700,10800,GOOG
3,2014-04-01,558.7100,568.4500,558.7100,567.1600,7900,GOOG
4,2014-04-02,565.1060,604.8300,562.1900,567.0000,146700,GOOG


In [11]:
master_df["ID"] = master_df["Ticker"]+master_df["Trading_Date"]

In [12]:
rds_connection_string = "root:GuyAzz42604260@localhost:3306/stocks_db"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')

In [13]:
engine.table_names()

['company_inform', 'trading_data']

In [29]:
try:
    stocks.to_sql(name='company_inform', con=engine, if_exists='append', index=False)
except Exception as e:
        print("Double check MySQL, do not duplicate your work.")

Double check MySQL, do not duplicate your work.


In [30]:
try:
    master_df.to_sql(name='trading_data', con=engine, if_exists='append', index=False)
except Exception as e:
        print("Plase double check your SQL database.")

Plase double check your SQL database.


In [ ]:
#def get_quote(ticker):
    #query_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol='+ticker.upper()+'&outputsize=full&apikey=BRTJEWEQ9KK7TK9G'
    #responses = requests.get(query_url).json()
    #res = pd.DataFrame(responses['Time Series (Daily)']).T
    #res.reset_index(inplace=True)
    #res.columns =['date','open','high','low','close','volume']
    #return {ticker.upper():res}

#tickers = ['GOOG']
#p = Pool(5)
#datas = p.map(get_quote, tickers)